In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime as dt
from bokeh.io import output_file, show
from bokeh.layouts import gridplot, layout
from bokeh.palettes import Category20
from bokeh.plotting import figure, curdoc
from bokeh.models import (ColumnDataSource, CDSView, RadioButtonGroup, GroupFilter, DataTable,
                          TableColumn, DateFormatter, CategoricalColorMapper, CheckboxGroup,
                          TextInput, Column, Row, Div, HoverTool, Slider, RangeSlider, MultiChoice)

In [2]:
# Read the data:
data = pd.read_csv('data/all_stocks_5yr.csv')
# Format the date variable to datetime and sort the data by date:
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data.sort_values(by='date', ascending=False, inplace=True)
# Create the month and year variables:
data['Month'] = pd.to_datetime(data['date']).dt.month
data['Year'] = pd.to_datetime(data['date']).dt.year

In [3]:
# Create a sorted list of ticker names:
tickers = sorted(list(data.Name.unique()))
# Initialize the ticker choice:
ticker_button = MultiChoice(value=tickers[:2], options=tickers)

In [4]:
# Creat a sorted list of years:
year = sorted(list(data.Year.unique()))
# Initialize the year slider:
year_slider = Slider(start=year[0], end=year[-1], value=year[1], step=1, title='Year')

In [5]:
# Creat a sorted list of months:
month = sorted(list(data.Month.unique()))
# Initialize the month range slider:
month_slider = RangeSlider(start=month[0], end=month[-1], value=month[:2], step=1, title='Month')

In [6]:
# Filter the initial data source:
df = data[(data['Name'].isin(ticker_button.value)) & (data['Month'] >= month_slider.value[0]) & (data['Month'] <= month_slider.value[1]) & (data['Year'] == year_slider.value)]
# Pass the filtered data source to the ColumnDataSource class:
source = ColumnDataSource(data=df)

In [7]:
def plot_function(tickers):
    # Getting some colors:
    colors = list(Category20.values())[17]
    random_colors = []
    for c in range(len(tickers)):
        random_colors.append(random.choice(colors))

    # Create the hovertool:
    TOOLTIPS = HoverTool(tooltips=[    ('date', '$x{%Y-%m-%d}'),
                   ('close', '$@{close}{0.0}'),
                   ('high', '$@{high}{0.0}'),
                   ('low', '$@{low}{0.0}'),
                   ('volume', '@volume{0.00 a}')],
                         formatters={'$x': 'datetime'})

    # Create the figure to store all the plot lines in:
    p = figure(x_axis_type='datetime', width=1000)
    
    # Loop through the tickers and colors and create plot line for each:
    for t, rc in zip(tickers, random_colors):
        view = CDSView(source=source, filters=[GroupFilter(column_name='Name', group=t)])
        p.line(x='date', y='close', source=source, view=view, line_color=rc, line_width=4)
    
    # Add the hovertool to the figure:
    p.add_tools(TOOLTIPS)
    return p
p = plot_function(tickers)

In [8]:
# Creating the list of columns:
columns = [
        TableColumn(field="date", title="Date", formatter=DateFormatter(format="%Y-%m-%d")),
        TableColumn(field="Name", title="Name"),
        TableColumn(field="close", title="Close"),
    ]
# Initializing the table:
table = DataTable(source=source, columns=columns)

In [9]:
def filter_function():
    # Filter the data according to the widgets:
    new_src = data[(data['Name'].isin(ticker_button.value)) & (data['Month'] >= month_slider.value[0]) & (data['Month'] <= month_slider.value[1]) & (data['Year'] == year_slider.value)]
    
    # Replace the data in the current data source with the new data:
    source.data = new_src.to_dict('series')

In [10]:
# Header
title = Div(text='<h1 style="text-align: center">Stock Dashboard</h1>')

In [11]:
widgets_col = Column(month_slider, year_slider)
widgets_row = Row(widgets_col, ticker_button)

In [12]:
layout = layout([[title],
                 [widgets_row],
                 [p,table]])

In [13]:
curdoc().title = 'Stock Dashboard'
curdoc().add_root(layout)